In [1]:
import bioMass.dataViz as dv
from bioMass.dataloader import SentinelDataset, SentinelDataModule
from bioMass.model import Unet
from pytorch_lightning import Trainer
import matplotlib.pyplot as plt
import torch
from torch.utils.data import DataLoader, random_split
from tqdm.notebook import tqdm
from torch import nn
import os
import numpy as np
import pandas as pd

from pytorch_lightning import loggers as pl_loggers

%config Completer.use_jedi = False
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [2]:
tensorboard_logdir = '/home/ubuntu/Thesis/backup_data/bioMass_data/model_logs/tryouts'

logger = pl_loggers.TensorBoardLogger(
    tensorboard_logdir, name="", default_hp_metric=False
)

trainer = Trainer(logger=logger, accelerator='gpu', max_epochs=2, devices=1, log_every_n_steps=1)
# trainer = Trainer(logger=logger, accelerator='gpu', max_epochs=1000, devices=1, log_every_n_steps=1, overfit_batches=1)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [3]:
loss_module = nn.MSELoss(reduction='mean')

dm = SentinelDataModule(loader_type='PCA', max_chips=100, loader_device='cpu', num_workers=1)
model = Unet(loss_fn=loss_module, lr=1e-4, init_n_features=64, in_channels=15, dropout_p=0.5)

trainer.fit(model, datamodule=dm)

/home/ubuntu/Thesis/miniconda3/envs/bioMass/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'loss_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_fn'])`.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]

   | Name    | Type       | Params
----------------------------------------
0  | maxpool | MaxPool2d  | 0     
1  | up      | Upsample   | 0     
2  | EB0     | Sequential | 45.6 K
3  | EB1     | Sequential | 221 K 
4  | EB2     | Sequential | 885 K 
5  | EB3     | Sequential | 3.5 M 
6  | Bottom  | Sequential | 4.7 M 
7  | DB2     | Sequential | 2.4 M 
8  | DB1     | Sequential | 589 K 
9  | DB0     | Sequential | 147 K 
10 | out     | Sequential | 73.9 K
11 | loss_fn | MSELoss    | 0     
----------------------------------------
12.6 M    Trainable params
0         Non-trainable params
12.6 M    Total params
50.331   

Sanity Checking: 0it [00:00, ?it/s]

/home/ubuntu/Thesis/miniconda3/envs/bioMass/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/ubuntu/Thesis/miniconda3/envs/bioMass/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 24 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


In [ ]:
SentinelDataModule(loader_type='PCA', max_chips=100, loader_device='cpu', num_workers=1)

In [25]:
dm_params = {'loader_type':'PCA', 'max_chips':100, 'loader_device':'cpu', 'num_workers':1}

In [27]:
SentinelDataModule(**dm_params).loader_type

'PCA'

In [24]:
getattr(dm, 'split_proportions')

[0.7, 0.2, 0.1]

In [5]:
model.hparams

"dropout_p":       0.5
"in_channels":     15
"init_n_features": 64
"loss_fn":         MSELoss()
"lr":              0.0001

In [11]:
new_hparams = model.hparams
new_hparams['new_param'] = 0

In [13]:
model.hparams['new_param'] = 'test'

In [19]:
model.save_hyperparameters()

/home/ubuntu/Thesis/miniconda3/envs/bioMass/lib/python3.9/site-packages/pytorch_lightning/utilities/parsing.py:262: UserWarning: Attribute 'loss_fn' is an instance of `nn.Module` and is already saved during checkpointing. It is recommended to ignore them using `self.save_hyperparameters(ignore=['loss_fn'])`.
  rank_zero_warn(


In [29]:
# update hyperparameters
import yaml

In [34]:
fname = os.path.join(model.logger.log_dir, 'hparams.yaml')

with open(fname) as f:
    newdct = yaml.load(f, Loader=yaml.BaseLoader)

print(newdct)

{'dropout_p': '0.5', 'in_channels': '15', 'init_n_features': '64', 'loss_fn': {'_backward_hooks': [[]], '_buffers': [[]], '_forward_hooks': [[]], '_forward_pre_hooks': [[]], '_is_full_backward_hook': 'null', '_load_state_dict_post_hooks': [[]], '_load_state_dict_pre_hooks': [[]], '_modules': [[]], '_non_persistent_buffers_set': {}, '_parameters': [[]], '_state_dict_hooks': [[]], 'reduction': 'mean', 'training': 'true'}, 'lr': '0.0001'}


In [35]:
newdct["dm_params"] = dm_params

In [37]:
with open(fname, "w") as f:
    yaml.dump(newdct, f)